In [4]:
import json
import os
from glob import glob
from tqdm import tqdm
from google import genai
from time import time

In [3]:
with open(f"{os.environ['HOME']}/secrets/apikeys.json", "r") as f:
    api_keys = json.load(f)

In [9]:
git_url = 'https://github.com/unclecode/crawl4ai.git'
git_url = 'https://github.com/microsoft/markitdown.git'
git_url = 'https://github.com/RockChinQ/LangBot.git'
git_url = 'https://github.com/meta-llama/llama-models.git'
git_url = 'https://github.com/ultralytics/yolov5.git'
git_url = 'https://github.com/garimasingh128/awesome-python-projects.git'
repo_name = git_url.split('/')[-1].split('.')[0]
repo_name

'awesome-python-projects'

In [10]:
!git clone $git_url 

Cloning into 'awesome-python-projects'...
remote: Enumerating objects: 1225, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
^Cceiving objects:  49% (605/1225), 93.45 MiB | 933.00 KiB/s 


In [5]:
pyfiles = glob(f"{repo_name}/**/*.py", recursive=True)
len(pyfiles)

51

In [6]:
sources = {}

for file in tqdm(pyfiles):
    with open(file) as f:
        sources[file] = f.read()
    

100%|██████████████████████████████████████████| 51/51 [00:00<00:00, 4785.98it/s]


In [7]:
nlines = {file: len([i for i in content.split('\n') if len(i.strip())>0 and not i.strip().startswith('#')]) for file,content in sources.items()}

In [8]:
sum(nlines.values()), len(nlines)

(14463, 51)

In [56]:
nlines;

In [58]:
prompt = """
Following you have the contents of a Python software project separated by source files.

Extract from all those files a list of functions and class methods that would require unit tests.

**FILES**

"""

prompt = """
Following you have the contents of a Python software project separated by source files.

Make a unit test for each significant functions or class methods that would require unit tests.

**FILES**

"""

files_str = ""
for k,v in sources.items():
    files_str += f"""
<SOURCE_FILE>
<FILE_PATH>{k}</FILE_PATH>
<FILE_CONTENT>
{v}
</FILE_CONTENT>
<SOURCE_FILE>

    """

prompt = prompt.replace("**FILES**", files_str)

In [60]:
len(prompt)

686181

In [61]:
from google import genai

client = genai.Client(api_key=api_keys['GEMINI_API_KEY'])


In [62]:
#model ="gemini-2.0-flash-001"
model ="gemini-2.5-pro-exp-03-25"



In [63]:
t0 = time()

cin = client.models.count_tokens(
    model = model,
    contents = prompt
)

count_tokens_elapsed_time = time() - t0
print (f"{cin}\nelapsed time {count_tokens_elapsed_time:.2f} secs")

total_tokens=193063 cached_content_token_count=None
elapsed time 1.70 secs


In [108]:
t0 = time()

response = client.models.generate_content(
    model = model,
    contents=prompt
)

generation_elapsed_time = time() - t0
print (f"elapsed time {generation_elapsed_time:.2f} secs")

total_tokens=161408 cached_content_token_count=None
elapsed time 248.77 secs


In [113]:
cout = client.models.count_tokens(
    model = model,
    contents = response.text
)
cout

CountTokensResponse(total_tokens=35907, cached_content_token_count=None)

In [110]:
len(response.text)

124022

In [ ]:
print(response.text)

In [114]:
with open('unittest.py', 'w') as f:
    f.write(response.text)